<a href="https://colab.research.google.com/github/PerceptronV/Apollo-Psi/blob/master/Haiku_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*Haiku GAN*

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Importing and Downloading Relevant Libraries; Data Processing

In [0]:
!pip install vibra --upgrade

In [0]:
print ('Apollo γ ------ initiating')

# Imp.d numpy [as np]; os; time; tensorflow [as tf]; future [absolute_import,division,print_function,unicode_literals]
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os
import time
import tensorflow as tf
import random
from tensorflow import keras
import vibra
from vibra import firefly as ff
tf.enable_eager_execution()

In [0]:
file = ff.file(filename='haikus.txt', src='https://raw.githubusercontent.com/PerceptronV/Apollo-sources/master/{Haikus}.txt')


char2idx = file.char2idx()
idx2char = file.idx2char()
vocab_size = file.char_size()
print (char2idx)
print (str(vocab_size))
print (char2idx['}'])

In [0]:
text=file.text()
text_as_list=text.split("}\n\n")
text_as_int=[]
for seg in text_as_list:
  text_as_int.append([char2idx[c] for c in seg])

max_length=0  

for asdf in text_as_int:
  if len(asdf) > max_length:
    max_length=len(asdf)

for lkjh in range(len(text_as_int)):
  #padding
  if len(text_as_int[lkjh]) < max_length:
    for qwer in range (max_length - len(text_as_int[lkjh])):
      text_as_int[lkjh].append(0)
  text_as_int[lkjh]=[text_as_int[lkjh]]

dataset=np.asarray(text_as_int)
print (dataset[0])
print (dataset.shape)

    
print (max_length)

## Making the Generator

### Making the seed generator

In [0]:
def gen_seed(vocab_size):
  
  seed=np.random.randint(1, size=(1,vocab_size))
      
  return (seed)

###Defining the Generator model

In [0]:
def make_generator (vocab_size,max_length):
  
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(32,input_shape=(vocab_size,)),
    keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(1024),
    keras.layers.Activation('exponential'),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(max_length),
  ])
  return model

'''def make_generator (vocab_size,max_length):
  
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 256,
                              batch_input_shape=[1, None]),
    rnn(1024,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=False),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(max_length)
  ])
  return model'''

In [0]:
'''Define text generation process
def gen_text (model,seed=None,training=False):
  
  if seed is None:
    seed=gen_seed()
    
  raw=[]

  temperature = 0.5

  model.reset_states()
  
  input_eval=seed
  
  while (1):
      predictions = model(input_eval,training=training)      
      ''''''
      predictions = tf.squeeze(predictions, 0)
      
      #print (predictions.shape)
      
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)
      place=idx2char[predicted_id]
      
      if place == '}':
        break
        
      raw.append(predicted_id)'''
      
      '''print (predictions.shape)
      predicted_id=np.argmax(predictions)'''
      
      
      

  
  '''raw=np.asarray(raw)    
  return (raw)'''

###Testing the untrained Generator

In [0]:
generator = make_generator(vocab_size,max_length)

In [0]:
txt=generator(gen_seed(vocab_size))

print (txt)

##Making the Discriminator

###Defining the Discriminator model

In [0]:
def make_discriminator (max_length):
  
  
  model = tf.keras.Sequential([
      
    tf.keras.layers.Dense(56,input_shape=(max_length,)),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation('sigmoid')])
  return model

'''def make_discriminator (vocab_size):
  
  
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 256),
    rnn(1024,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',),
      
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation('sigmoid')])
  return model'''

In [0]:
'''def dis_text (model,inp,training=False):
  

  inp=tf.expand_dims(inp,0)
  model.reset_states()
  
  dis = model(inp,training=training)
  dis=tf.squeeze(dis,[0,2])
  out=dis[len(dis)-1]
  out=tf.reshape(out,[1,1])
  
  return (out)'''

###Testing the untrained Discriminator

In [0]:
discriminator = make_discriminator(max_length)
a = discriminator(txt)
print(a)

##Defining the losses 

###Global loss function

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

###Discriminator loss

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [0]:
print (discriminator_loss(discriminator(dataset[0]),a))

###Generator loss

In [0]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
print(generator_loss(a))

##Defining the training process

###Defining the optimizers

In [0]:
generator_optimizer = tf.train.AdamOptimizer()
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

###Saving Checkpoints

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

###Initializing the training process variables

In [0]:
EPOCHS = 50
seed = gen_seed(vocab_size)

###Defining one training step

In [0]:
@tf.function
def train_step(inp):
     
    for tyui in range (10):
      
      with tf.GradientTape() as disc_tape:
      
        generated_text = generator(gen_seed(vocab_size))

        real_output = discriminator(inp)
        fake_output = discriminator(generated_text)
        
        disc_loss = discriminator_loss(real_output, fake_output)
        
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    for tyui in range (10):
      
      with tf.GradientTape() as gen_tape:

        generated_text = generator(gen_seed(vocab_size))
        
        fake_output = discriminator(generated_text)

        gen_loss = generator_loss(fake_output)

      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return gen_loss,disc_loss

###Defining the training process

In [0]:
def train(vocab_size,dataset, epochs):
  for epoch in range(epochs):
    
    start = time.time()
    
    for i in dataset:
      g,d=train_step(i)
    
    print ("EPOCH {}:".format(epoch + 1))
    
    out=generator(seed)
    print (out)

    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec\nGenerator loss: {}\nDiscriminator loss: {}'\
           .format(epoch + 1, time.time()-start,g,d))
    
    out=generator(seed)
  
    fout=''

    for i2 in out[0]:
      if round(int(i2))<vocab_size and round (int(i2)) > 0:
        fout+=str(idx2char[round(int(i2))])

    print ('\n'+fout+'\n\n\n')

  
  print ("\nAfter {} epochs:".format(epochs))
  out=generator(seed)
  
  fout=''

  for i2 in out[0]:
    if round(int(i2))<vocab_size and round (int(i2)) > 0:
      fout+=str(idx2char[round(int(i2))])
  print ('\n'+fout+'\n\n\n')

###Training

In [0]:
%%time
train(vocab_size, dataset, EPOCHS)

##Main Source

In [0]:
print ('Apollo γ ------ initiating\n\n')

!pip install vibra --upgrade
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os
import time
import tensorflow as tf
import random
from tensorflow import keras
import vibra
from vibra import firefly as ff
tf.enable_eager_execution()

def gen_seed(vocab_size):
  
  seed=np.random.randint(1, size=(1,vocab_size))
      
  return (seed)

def make_generator (vocab_size,max_length):
  
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(32,input_shape=(vocab_size,)),
    keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(1024),
    keras.layers.Activation('exponential'),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(max_length),
  ])
  return model

def make_discriminator (max_length):
  
  
  model = tf.keras.Sequential([
      
    tf.keras.layers.Dense(56,input_shape=(max_length,)),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation('sigmoid')])
  return model

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.train.AdamOptimizer()
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
  
@tf.function
def train_step(inp):
     
    for tyui in range (10):
      
      with tf.GradientTape() as disc_tape:
      
        generated_text = generator(gen_seed(vocab_size))

        real_output = discriminator(inp)
        fake_output = discriminator(generated_text)
        
        disc_loss = discriminator_loss(real_output, fake_output)
        
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    for tyui in range (10):
      
      with tf.GradientTape() as gen_tape:

        generated_text = generator(gen_seed(vocab_size))
        
        fake_output = discriminator(generated_text)

        gen_loss = generator_loss(fake_output)

      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return gen_loss,disc_loss
  
def train(vocab_size,dataset, epochs):
  for epoch in range(epochs):
    
    start = time.time()
    
    for i in dataset:
      g,d=train_step(i)
    
    print ("EPOCH {}:".format(epoch + 1))
    
    out=generator(seed)
    print (out)

    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec\nGenerator loss: {}\nDiscriminator loss: {}'\
           .format(epoch + 1, time.time()-start,g,d))
    
    out=generator(seed)
  
    fout=''

    for i2 in out[0]:
      if round(int(i2))<vocab_size and round (int(i2)) > 0:
        fout+=str(idx2char[round(int(i2))])

    print ('\n'+fout+'\n\n\n')

  
  print ("\nAfter {} epochs:".format(epochs))
  out=generator(seed)
  
  fout=''

  for i2 in out[0]:
    if round(int(i2))<vocab_size and round (int(i2)) > 0:
      fout+=str(idx2char[round(int(i2))])
  print ('\n'+fout+'\n\n\n')

file = ff.file(filename='haikus.txt', src='https://raw.githubusercontent.com/PerceptronV/Apollo-sources/master/{Haikus}.txt')

char2idx = file.char2idx()
idx2char = file.idx2char()
vocab_size = file.char_size()

text=file.text()
text_as_list=text.split("}\n\n")
text_as_int=[]

for seg in text_as_list:
  text_as_int.append([char2idx[c] for c in seg])

max_length=0  

for asdf in text_as_int:
  if len(asdf) > max_length:
    max_length=len(asdf)

for lkjh in range(len(text_as_int)):
  if len(text_as_int[lkjh]) < max_length:
    for qwer in range (max_length - len(text_as_int[lkjh])):
      text_as_int[lkjh].append(0)
  text_as_int[lkjh]=[text_as_int[lkjh]]

dataset=np.asarray(text_as_int)

generator = make_generator(vocab_size,max_length)

discriminator = make_discriminator(max_length)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

EPOCHS = 50
seed = gen_seed(vocab_size)

In [0]:
%%time
train(vocab_size, dataset, EPOCHS)